In [18]:
import os
import time
import openpyxl
import pandas as pd
from typing import List
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc



load_dotenv()

True

In [19]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            logger.info('Conexão estabelecida!')
            pass
        logger.info('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        logger.info(f"Error: {e}")
        return None

In [20]:
def generate_sql_query() -> List[str]:
    current_month = f"{datetime.now().month:02}"
    current_year = datetime.now().year % 100  # Get the last two digits of the year

    last_month = f"{(datetime.now().month - 1) % 12:02}"
    last_year = current_year if datetime.now().month > 1 else current_year - 1

    tables = [f"movprodd{last_month}{last_year}", f"movprodd{current_month}{current_year}"]

    return tables

print(generate_sql_query())

['movprodd0823', 'movprodd0923']


In [27]:
conn = get_db_engine() 

def vendas_query(conn):
    query = (f"""
        SELECT
            prun.prun_unid_codigo AS unidade,
            prun.prun_ativo as tipo,
            prun.prun_prod_codigo::TEXT AS prod_codigo,
            prod.prod_codbarras AS cod_barras,
            prod.prod_marca AS marca,
            TO_CHAR(prod.prod_codigo, '00999') AS cod_prod,
            forn.forn_razaosocial AS razao_fornecedor,
            forn.forn_cnpjcpf AS cnpj_fornecedor,
            prun.prun_emb AS embalagem,
            prod.prod_descricao AS produto,
            prun.prun_estoque1 AS estoque1,
            prod.prod_status AS status_prod,
            prod.prod_forn_codigo::TEXT AS cod_fornecedor
            FROM produn AS prun 
            LEFT JOIN produtos AS prod ON prun.prun_prod_codigo = prod.prod_codigo
            LEFT JOIN fornecedores AS forn ON prod.prod_forn_codigo = forn.forn_codigo
            WHERE prun.prun_bloqueado = 'N' 
            AND prun.prun_unid_codigo = '001'
            AND prun.prun_ativo = 'S'
            AND prod.prod_marca IN ('BRF','BRF IN NATURA')
            AND prod.prod_status = 'N'
    """)
        # AND mprd.mprd_datamvto > '2023-01-09'
    
    df = pd.read_sql_query(query, conn)
    return df

display(vendas_query(conn))

2023-09-29 13:55:14.500 | INFO     | __main__:get_db_engine:7 - Conexão estabelecida!
2023-09-29 13:55:14.500 | INFO     | __main__:get_db_engine:9 - Banco de dados conectado!


,unidade,tipo,prod_codigo,cod_barras,marca,cod_prod,razao_fornecedor,cnpj_fornecedor,embalagem,produto,estoque1,status_prod,cod_fornecedor
0,001,S,1049,7891515977419,BRF,01049,BRF S.A.,01838723010513,KG,CORACAO DE FGO PCT PERDIGAO CX16KG,112.000,N,3404
1,001,S,22870,7891515606329,BRF,22870,BRF S.A.,01838723010513,UN,MAC'N CHEESE TRADICIONAL 300G SADIA CX9UN,99.000,N,3404
2,001,S,23329,7893000444904,BRF,23329,BRF S.A.,01838723010513,KG,APRESUNTADO RETANGULAR SADIA 3.5KG,187.900,N,3404
3,001,S,8788,7891515496234,BRF,08788,BRF S.A.,01838723010513,UN,LASANHA PERDIGAO DE FRANGO 600GR CX6UN,247.000,N,3404
4,001,S,18937,7891515919313,BRF,18937,BRF S.A.,01838723010513,KG,LING. TOSCANA APIMENTADA NA BRASA PERDIGAO PCT...,79.000,N,3404
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,001,S,9989,7891515742034,BRF,09989,BRF S.A.,01838723010513,UN,SALSICHA PERDIGAO PCT 500G CX12UN,126.000,N,3404
214,001,S,19046,7891515767601,BRF,19046,BRF S.A.,01838723010513,KG,BACON MANTA PERDIGAO CX12KG,816.505,N,3404
215,001,S,17884,7891515490218,BRF,17884,BRF S.A.,01838723010513,UN,FAT. SALAME DEFUMADO PERDIGAO 100GR CX30UN,604.000,N,3404
216,001,S,8842,7891515605797,BRF,08842,BRF S.A.,01838723010513,CX,HAMBURGUER MISTO PERDIGAO 36X56G,168.000,N,3404
